# **Demo:** Net2WiderNet on MNIST with LeNet5

The following demo shows how to apply Net2WiderNet to LeNet5 in order to increase the number of output filters of a convolutional layer. The input image shape is the one of MNIST, but the network and the Net2WiderNet algorithm can be applied to any other image size.

In [1]:
# Import libraries
import torch
import torchinfo

# Import custom modules and packages
from lenet import LeNet
import params.lenet_mnist
import net2net.net2net_wider

### 1. Create a LeNet5 model

We start by creating the standard LeNet5 model.

In [2]:
# Create a LeNet model
model = LeNet(nb_classes=params.lenet_mnist.NB_CLASSES)

# Create a random input
x = torch.randn(1,
                params.lenet_mnist.NB_CHANNELS,
                *params.lenet_mnist.IMAGE_SHAPE)

# Compute the output of the teacher network
y_teacher = model(x)

### 2. Create a wider version of LeNet5

We then apply the Net2WiderNet algorithm to the standard LeNet5 model to increase the number of output filters of the first convolutional layer. The weights and biases of the student model are initialized with those of the teacher model, in such a way that the output of the student model is the same as the output of the teacher model for the same input at initialization.

In [3]:
# Instantiate a Net2Net object from a (pre-trained) model
net2net = net2net.net2net_wider.Net2Net(teacher_network=model)

# Set the widening operations to be performed
# Here we only increase the width of the first convolutional layer
wider_operations = {"operation1": {"target_conv_layers": ["layer1.0"],
                    "width": [10],
                    "batch_norm_layers": ["layer1.1"],
                    "next_layers": ["layer2.0"]}}

# Add some noise to the copied weights (optional)
sigma = 0.  # Standard deviation of the noise


# Go through the list of widening operations
for key in wider_operations.keys():

    print("Widening operation: ", key)
    
    # Get the parameters of the wider operation
    target_conv_layers = wider_operations[key]["target_conv_layers"]
    next_layer = wider_operations[key]["next_layers"]
    new_width = wider_operations[key]["width"]
    batch_norm_layers = wider_operations[key]["batch_norm_layers"]

    # Widen a layer of the network
    net2net.net2wider(target_conv_layers=target_conv_layers,
                      next_layers=next_layer,
                      width=new_width,
                      batch_norm_layers=batch_norm_layers,
                      sigma=sigma)


# Compute the output of the student network
y_student = net2net.student_network(x)

Widening operation:  operation1


### 3. Check that the student and teacher models have the same output for the same input

We check that the output of the student model is the same as the output of the teacher model for the same input at initialization. They can be slightly different if some noise has been added to the weights of the student model during the initialization.

In [4]:
# The outputs should be the same
print("Teacher output: ", y_teacher)
print("Student output: ", y_student, "\n")

Teacher output:  tensor([[ 0.0497,  0.1230,  0.2026, -0.0270,  0.0548,  0.0403,  0.0751,  0.0564,
          0.1223,  0.0191]], grad_fn=<AddmmBackward0>)
Student output:  tensor([[ 0.0497,  0.1230,  0.2026, -0.0270,  0.0548,  0.0403,  0.0751,  0.0564,
          0.1223,  0.0191]], grad_fn=<AddmmBackward0>) 



### 4. Have a look at the student and teacher architectures

We display the student and teacher architectures to check that the student model has more filters than the teacher model in the first convolutional layer.

In [5]:
# Display the architecture of the student network
torchinfo.summary(model, input_size=(1,
                                     params.lenet_mnist.NB_CHANNELS,
                                     *params.lenet_mnist.IMAGE_SHAPE))

Layer (type:depth-idx)                   Output Shape              Param #
LeNet                                    [1, 10]                   --
├─Sequential: 1-1                        [1, 6, 14, 14]            --
│    └─Conv2d: 2-1                       [1, 6, 28, 28]            156
│    └─BatchNorm2d: 2-2                  [1, 6, 28, 28]            12
│    └─ReLU: 2-3                         [1, 6, 28, 28]            --
│    └─MaxPool2d: 2-4                    [1, 6, 14, 14]            --
├─Sequential: 1-2                        [1, 16, 5, 5]             --
│    └─Conv2d: 2-5                       [1, 16, 10, 10]           2,416
│    └─BatchNorm2d: 2-6                  [1, 16, 10, 10]           32
│    └─ReLU: 2-7                         [1, 16, 10, 10]           --
│    └─MaxPool2d: 2-8                    [1, 16, 5, 5]             --
├─Linear: 1-3                            [1, 120]                  48,120
├─ReLU: 1-4                              [1, 120]                  --
├─Linea

In [6]:
# Display the architecture of the student network
torchinfo.summary(net2net.student_network, input_size=(1,
                                                       params.lenet_mnist.NB_CHANNELS,
                                                       *params.lenet_mnist.IMAGE_SHAPE))

Layer (type:depth-idx)                   Output Shape              Param #
LeNet                                    [1, 10]                   --
├─Sequential: 1-1                        [1, 10, 14, 14]           --
│    └─Conv2d: 2-1                       [1, 10, 28, 28]           260
│    └─BatchNorm2d: 2-2                  [1, 10, 28, 28]           20
│    └─ReLU: 2-3                         [1, 10, 28, 28]           --
│    └─MaxPool2d: 2-4                    [1, 10, 14, 14]           --
├─Sequential: 1-2                        [1, 16, 5, 5]             --
│    └─Conv2d: 2-5                       [1, 16, 10, 10]           4,016
│    └─BatchNorm2d: 2-6                  [1, 16, 10, 10]           32
│    └─ReLU: 2-7                         [1, 16, 10, 10]           --
│    └─MaxPool2d: 2-8                    [1, 16, 5, 5]             --
├─Linear: 1-3                            [1, 120]                  48,120
├─ReLU: 1-4                              [1, 120]                  --
├─Linea